In [1]:
base_path = '/home/datasets/rsna-intracranial-hemorrhage-detection/'
csv_path = base_path + 'stage_2_train.csv'
items = base_path + 'stage_2_train/'

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch, csv, pydicom
import torchvision.transforms as transforms
from PIL import Image

class RSNASet(Dataset):
    def __init__(self, csv_path, root_dir):
        self.root_dir = root_dir
        labels = {}
        with open(csv_path) as f:
            rdr = csv.reader(f)
            next(rdr)
            for rows in rdr:
                fn = '_'.join(rows[0].split('_')[0:2])
                try:
                    if len(labels[fn]) == 6:
                        continue
                    labels[fn].append(int(rows[1]))
                except KeyError:
                    labels[fn] = [int(rows[1])]
        for fn in labels.keys():
            # "any" should really be the classification "none"
            # so, it should only be 1 if nothing else is true - swap the values
            labels[fn][5] = abs(labels[fn][5]-1)
        self.labels = labels
        self.files = list(labels.keys())
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        file = self.files[idx]
        label = torch.argmax(torch.FloatTensor(self.labels[file]))
        img = pydicom.dcmread(self.root_dir+file+'.dcm').pixel_array
        tf = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((512, 512)),
            transforms.Lambda(lambda x: x.repeat(3,1,1)),
            transforms.ConvertImageDtype(torch.float),
        ])
        img = tf(Image.fromarray(img))
        return img, label

In [3]:
tempset = RSNASet(csv_path=csv_path, root_dir=items)

In [4]:
import numpy as np
import torch.nn.functional as F
vals = list(tempset.labels.values())
t = np.argmax(vals, axis=1)
counts = torch.FloatTensor(np.bincount(t))
assert counts.sum() == len(tempset)
counts = counts / counts.sum()

In [5]:
print(counts)

tensor([0.0042, 0.0471, 0.0207, 0.0286, 0.0428, 0.8566])


In [6]:
BATCH_SIZE = 16

import math
from focal import FocalLoss
trainlen = math.floor(0.80 * len(tempset)) # 80/20 train/val split
vallen = len(tempset) - trainlen
# trainlen = 512
# vallen = 256

trainset, valset = torch.utils.data.random_split(tempset, [trainlen, vallen], generator=torch.Generator().manual_seed(42))
model = 'resnet50'
trainloader = torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,num_workers=2, shuffle=True)
valloader = torch.utils.data.DataLoader(valset,batch_size=BATCH_SIZE,num_workers=2, shuffle=True)
classes = ('epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural','none')
model = torch.hub.load('pytorch/vision:v0.10.0', model, pretrained=False, num_classes=len(classes))
if torch.cuda.is_available():
    print("CUDA is available - using CUDA!")
    model = model.cuda()
    counts = counts.cuda()
resume = False
old_epoch = 0
if resume:
    print("Resuming training!")
    old_epoch = 0
    model_load = 'weights/split_resnet50/epoch{}.pth'.format(old_epoch)
    if not torch.cuda.is_available():
        model.load_state_dict(torch.load(model_load, map_location=torch.device('cpu')))
    else:
        model.load_state_dict(torch.load(model_load))
import torch.optim as optim
import torch.nn as nn
criterion = nn.CrossEntropyLoss(weight=counts)
optimizer = optim.Adam(model.parameters(), lr=0.011)

Using cache found in /home/dsingh/.cache/torch/hub/pytorch_vision_v0.10.0


CUDA is available - using CUDA!


In [7]:
from tqdm import tqdm
for epoch in range(20):  # loop over the dataset multiple times
    epoch += old_epoch
    with tqdm(trainloader, unit="batch") as tepoch:
        model.train()
        running_loss = 0.0
        batch = 0
        total_correct = 0
        total = 0
        for inputs, labels in tepoch:
            tepoch.set_description(f"Epoch {epoch + 1}")
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            predictions = outputs.argmax(dim=1, keepdim=True).squeeze()
            correct = (predictions == labels).sum().item()
            total_correct += correct
            total += BATCH_SIZE
            accuracy = total_correct / total # batch_size
            running_loss += loss.item()
            if batch > 0:
                tepoch.set_postfix(loss=running_loss/batch, accuracy=100. * accuracy)
            batch += 1
    with tqdm(valloader, unit="batch") as tepoch:
        model.eval()
        running_loss = 0.0
        batch = 0
        total_correct = 0
        total = 0
        for inputs, labels in tepoch:
            tepoch.set_description(f"Validating: Epoch {epoch + 1}")
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            predictions = outputs.argmax(dim=1, keepdim=True).squeeze()
            correct = (predictions == labels).sum().item()
            total_correct += correct
            total += BATCH_SIZE
            accuracy = total_correct / total # batch_size
            running_loss += loss.item()
            if batch > 0:
                tepoch.set_postfix(loss=running_loss/batch, accuracy=100. * accuracy)
            batch += 1

    print('Saving model at epoch {}'.format(epoch + 1))
    torch.save(model.state_dict(), "weights/split_resnet50/epoch{}.pth".format(epoch + 1))
    print('Saved model at epoch {}!'.format(epoch + 1))
print('Finished training!')

Epoch 1:   0%|          | 0/9411 [00:08<?, ?batch/s]


RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 7.93 GiB total capacity; 7.03 GiB already allocated; 148.38 MiB free; 7.05 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF